# Flask 2: БД и веб-формы

Элементы, которые полезно уметь использовать при создании сайта

1. [Классы](https://github.com/hse-ling-python/seminars/blob/master/classes/classes.ipynb)
2. [CSS](https://github.com/hse-ling-python/seminars/blob/master/html_and_requests/html_css.ipynb)
3. [HTML, CSS, Bootstrap](https://github.com/hse-ling-python/seminars/blob/master/flask_applications/html_css_web_forms_2024.ipynb)

## Веб-формы

Часто нужно не просто показать страницы, но и получить от пользователя какую-то информацию, которую можно получить с помощью веб-форм.

```
<form>
.
элементы формы
.
</form>
```

Элементы - это input или select элементы, которые позволяют получить определенные типы информации

### Input 

```
<input type="submit" value="Submit">
```
- type - тип данных. Виды: text (текст), textarea (длинный текст), checkbox (выбор нескольких), radio (выбор одного), 
- submit - отправка формы,
- name - имя элемента при отправке формы (ключ, по которому можно будет потом получить значение),
- value - дефолтное значение.


Пример:
```
<input type="text" name="firstname" value="MyName">
```
- тип данных - текст
- имя параметра - firstname
- дефолтное значение - MyName

В текстовом поле дефолтное значение, если его не изменить, отправится в форме. Если мы хотим показать какое-то значени в ячейке (в качестве подсказки), но не отправлять, можно использовать параметр *placeholder*:
```
<form action="/result">
  First name:<br>
  <input type="text" name="firstname" placeholder="Mickey">
  <br>
  Last name:<br>
  <input type="text" name="lastname" value="Mouse">
  <br><br>
  <input type="submit" value="Отправить">
</form> 
```
Такая форма отправит запрос "firstname=&lastname=Mouse" при нажатии на кнопку "Отправить", если мы не изменим значения полей, потому что в первом случае Mickey - это placeholder, а Mouse - value.



Для выбора одного варианта из предложенных используется тип radio. Дефолтное значение задается параметром checked. Заметим, что отображаться будут Male, Female и тд, но при отправке запроса будем получать "gender=male", как указано в value:
```
<form>
  <input type="radio" name="gender" value="male" checked> Male<br>
  <input type="radio" name="gender" value="female"> Female<br>
  <input type="radio" name="gender" value="other"> Other
</form>
```

Следующий вид - это чекбоксы, то есть поля, которые мы отмечаем или не отмечаем галочкой:
```
<input type="checkbox" name="student" value="is_student"> студент<br>
```
Здесь при отправке получим "student=is_student", если отметим поле, иначе -- "".


Чтобы передать полученные данные нашему серверу, мы должны из отправить по какому-то адресу, например, results и сделать это в отдельной вкладке
```
<form action="/results" target="_blank">
```
- action - куда отправить
- target = "_blank" - сделать это в новой вкладке

### Методы обработки запроса: GET, POST

[Подробнее](https://code.luasoftware.com/tutorials/flask/flask-get-request-parameters-get-post-and-json)

При отправке запроса нужно сказать серверу, как ему стоит обаботать наш запрос -- каким методом. Методы бывают разные, а сейчас мы разберем два основных: GET и POST.

GET

Метод предназначен для получения требуемой информации и передачи данных в адресной строке. Пары «имя=значение» присоединяются в этом случае к адресу после вопросительного знака и разделяются между собой амперсандом (&). Удобство использования метода GET заключается в том, что адрес со всеми параметрами можно использовать неоднократно, сохранив его, например, в закладки браузера, а также менять значения параметров прямо в адресной строке.

POST

Метод POST посылает на сервер данные в запросе браузера. Это позволяет отправлять большее количество данных, чем доступно методу GET, плюс эти данные можно скрывать. Большие объемы данных используются в форумах, почтовых службах, заполнении базы данных, при пересылке файлов и др.

```
<form action="/results" method ='GET'>
```

### Пример html-файла формы
```
<html>
    <head>
       <meta charset="utf-8">
       <title>Что ищем?</title>
    </head>
    <body>
       <form action="https://www.google.com/search">
          <p><b>Введите поисковый запрос</b></p>
          <input type="text" name="q">
          <p><input type="submit"></p>
       </form>
     </body>
 </html>
 ```

## Интеграция БД: Flask-SQLalchemy

Для того, чтобы удобно было работать с базами в обычных программах (особенно в приложениях), придумали **ORM** (Object-Relational Mapping, объектно-реляционное отображение или преобразование) - это специальный инструмент для перевода объектов из БД в удобную форму в языке программирования (например, в классы питона). 

Мы уже встречали классы pymorphy, где разбор имел разные атрибуты (которые вызывались через точку) и внутри них значения или более сложная структура (то есть .tag имел еще .tag.POS и т.д.).

### Case-study

Предположим, что мы хотим сделать лингвистическую анкету и у нас есть такая вводная информация:

Мы хотим:
- собрать информацию про информанта (гендер, уровень образования и возрас),
- задать информанту два вопроса на оценку предложений,
- куда-то сохранять результаты.


Положим всё это в БД. Создадим таблицы по следующей схеме:

**User**
- id
- gender
- education
- age

**Questions**
- id
- text

**Answers**
- id (соответсвует id пользователя)
- q1 (ответ на первый вопрос)
- q2 (ответ на второй вопрос)

In [1]:
import sqlite3

db = sqlite3.connect('test.db')
cur = db.cursor()

In [2]:
cur.execute(
    """CREATE TABLE answers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    q1` INTEGER,
    q2` INTEGER )
    """)

cur.execute(
    """CREATE TABLE questions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    text TEXT
    )""")

cur.execute(
    """CREATE TABLE 
    user ( 
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    gender TEXT,
    education TEXT,
    age INTEGER )""")

db.commit()

### Как подключить базу к приложению?

Модуль flask-sqlalchemy позволяет работать с БД из flask-приложения. Необходимо прописать путь к БД: там есть часть ```sqlite3:///```- это обозначение того, что мы работает с таким типом базы,  дальше следует путь внутри проекта.

In [5]:
#!pip install flask_sqlalchemy
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///test.db'
db = SQLAlchemy(app)

### Как передать устройство своей базы питону?

Нам необходимо объяснить питону, где какие типы данных у нас лежат, для этого нам нужно описать каждую таблицу. Это можно сделать при помощи классов.

Не будем пока углубляться в классы питона, просто делаем по образцу.

Каждая таблица описывается отделно, с помощью ```__tablename__``` передается ее имя, а названия столбиков совпадают с названиями в нашей таблице. Еще используется специальная "обертка" для столбца, где можно прописать, какой это типа данных и если это первиный ключ, то на это указать

In [6]:
class User(db.Model):
    __tablename__ = 'user'  # имя таблицы
    id = db.Column(db.Integer, primary_key=True) # имя колонки = специальный тип (тип данных, первичный ключ)
    gender = db.Column(db.Text)
    education = db.Column(db.Text)
    age = db.Column(db.Integer)


class Questions(db.Model):
    __tablename__ = 'questions'
    id = db.Column(db.Integer, primary_key=True)
    text = db.Column(db.Text)


class Answers(db.Model):
    __tablename__ = 'answers'
    id = db.Column(db.Integer, primary_key=True)
    q1 = db.Column(db.Integer)
    q2 = db.Column(db.Integer)

### Как читать из базы (простые вещи)?

Простейшие запросы можно делать прямо к User, Questions, Answers. Например, мы хотим брать предложения для оценки информантов из базы.

Мы должны:

1. получить эти вопросы,
2. передать на страницу,
3. пройти по вопросам и сгенерировать анкету.

Создадим путь ```/questions``` и там считаем вопросы из базы и передадим на страницу через ```render_template```.

In [ ]:
from flask import render_template

@app.route('/questions')
def question_page():
    questions = Questions.query.all() # имя_таблицы.query.взять_все(), можно взять одну запись: .get(1)
    return render_template(
        'questions.html',
        questions=questions
    )

А как теперь это использовать?

Предположим, что у нас есть N вопросов (2 в нашем случае) и они одинаково устроены. Мы можем просто сгенерировать места для ответов и они все по очереди будут выводиться.

```{{question.text}}``` - выводит нам поле text из вопроса
```name="q{{ question.id }}"``` - использует id вопроса, чтобы получить имена для элементов например, ```name="q1"``` (чтобы мы могли потом достать ответ именно на первый вопрос)

```
{% for question in questions %}
    <div class="row">
        <p class="col-md-6">{{question.text}}</p>
        <table class="col-md-8">
            <tr>
                <td><input class="radio" type="radio" name="q{{ question.id }}" value=5></td>
                <td><input class="radio" type="radio" name="q{{ question.id }}" value=4></td>
                <td><input class="radio" type="radio" name="q{{ question.id }}" value=3></td>
                <td><input class="radio" type="radio" name="q{{ question.id }}" value=2></td>
                <td><input class="radio" type="radio" name="q{{ question.id }}" value=1></td>
            </tr>
            <tr>
                <td>отлично</td>
                <td>хорошо</td>
                <td>норм</td>
                <td>плохо</td>
                <td>ужасно</td>
            </tr>
        </table>
    </div>
{% endfor %}
```

### Как писать в базу?

Мы указали в форме ```<form action="/process">```, чтобы данные отправлялись на путь *process* и мы там будем их обрабатывать:


1. Достать все параметры из адреса с GET параметрами (типа ```gender=female&education=hse```) 
2. Записать в базу
3. Сохранить
4. Сказать пользователю, что все ок
5. Если пришел пустой запрос, то отправить проходить анкету

In [ ]:
from flask import request, redirect, url_for

@app.route('/process', methods=['get'])
def answer_process():
    
    # если пустой запрос, то отправить проходить анкету
    if not request.args:
        return redirect(url_for('question_page'))
    
    # получаем значения ответов
    gender = request.args.get('gender')
    education = request.args.get('education')
    age = request.args.get('age')
    
    # записываем в базу
    user = User(
        age=age,
        gender=gender,
        education=education
    )
    db.session.add(user)
    db.session.commit()
    
    # обновляем user'a, чтобы его ответ записать с таким же id
    db.session.refresh(user)
    
    # это же делаем с ответом
    q1 = request.args.get('q1')
    q2 = request.args.get('q2')
    answer = Answers(
        id=user.id,
        q1=q1,
        q2=q2
    )
    db.session.add(answer)
    db.session.commit()
    
    return 'Ok' # пользователь попадает на страницу, где написано только Ок

### Как получить из базы что-то сложное?

Нам нужна какая-то статистика на сайт, чтобы любопытные могли посмотреть сколько человек уже прошли анкету и какие примерно ответы получаются

In [10]:
from sqlalchemy import func # это позволит использовать функции типа среднее, максимум, минимум и т.д.

In [11]:
@app.route('/stats')
def stats():
    # заводим словарь для значений (чтобы не передавать каждое в render_template)
    all_info = {}
    
    age_stats = db.session.query(
        func.avg(User.age), # средний возраст AVG(user.age)
        func.min(User.age), # минимальный возраст MIN(user.age)
        func.max(User.age)  # максимальный возраст MAX(user.age)
    ).one() # берем один результат (он всего и будет один)
    
    all_info['age_mean'] = age_stats[0]
    all_info['age_min'] = age_stats[1]
    all_info['age_max'] = age_stats[2]
    
    # это простой запрос, можно прямо у таблицы спросить
    all_info['total_count'] = User.query.count() # SELECT COUNT(age) FROM user
    
    # SELECT AVG(q1) FROM answers
    all_info['q1_mean'] = db.session.query(func.avg(Answers.q1)).one()[0]
    
    # SELECT q1 FROM answers
    q1_answers = db.session.query(Answers.q1).all()
    
    return render_template('results.html', all_info=all_info)